<a href="https://colab.research.google.com/github/neerajgoud23/Triton-CUDA/blob/main/Day_1_(Triton_Vector_ADD).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install triton


In [3]:
import torch

In [5]:
import triton
import triton.language as tl

In [12]:
@triton.jit
def add_kernel(x_ptr,y_ptr,output_ptr,n_elements, BLOCK_SIZE: tl.constexpr):
  pid = tl.program_id(axis=0)
  block_start = pid*BLOCK_SIZE
  offsets = block_start +  tl.arange(0,BLOCK_SIZE)
  mask = offsets < n_elements
  x_val = tl.load(x_ptr + offsets,mask=mask)
  y_val = tl.load(y_ptr + offsets , mask=mask )
  output = x_val + y_val
  tl.store(output_ptr + offsets,output,mask=mask)

In [15]:
def add(x:torch.Tensor , y:torch.tensor):
  output = torch.empty_like(x)
  assert x.is_cuda and y.is_cuda and output.is_cuda
  n_elements = output.numel()
  BLOCK_SIZE = 1024
  grid = lambda meta : (triton.cdiv(n_elements , BLOCK_SIZE),)
  add_kernel[grid](x,y,output,n_elements,BLOCK_SIZE= BLOCK_SIZE)
  return output


In [16]:
x = torch.rand(98432,device='cuda')
y = torch.rand(98432,device='cuda')

output = add(x,y)
print(output)

tensor([1.0365, 0.9815, 0.8651,  ..., 0.1581, 1.4386, 0.7042], device='cuda:0')
